### cars_price1是处理了一些不要的特征，把一些特征做了变换比如邮编和used_time，删除了一些特征和异常值
第一部分的时间特征直接计算了，还可分开计算月份（月份也影响买车价格，可以将其进行分桶）
### 这一部分进行特征构造，并建立模型进行训练
类别特征：进行特征编码（目标编码（或者聚类one-hot编码））、统计特征构造、交叉组合；embedding也是一种思路

数值特征：分桶（还得进行目标编码）、加减组合、多项式组合、统计特征（平均值、总和、方差等）

特征选择：filter、wrapper、Embedded（pearson、RFE（递归特征消除）、单因素分析、随机森林、xgb、lgb、线性回归、岭回归、套索回归等）

归一化处理：数值归一化MinMaxScaler/StanderScaler/log1p/

target:处理用对数、box-cox等（本文用box-cox）
最新成绩MAE：540（只是按照上述思路做了特征的70%，模型参数和融合都没有做）

In [10]:
# 引入一些包，随便引入不够再加
import pandas as pd 
import numpy as np
%matplotlib inline
import numpy as np 
import pandas as pd 
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as st
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# 利用GPU训练：有无效果没有测试
import os
os.environ["CUDA_VISIBLE_DEVICES"] = '1'

In [11]:
# 读入处理后的CSV文件
path = 'I:/old_cars/EDA/'
# Train_data = pd.read_csv(path + 'used_car_train_20200313.csv', sep=' ')
df = pd.read_csv(path + 'pre_data.csv')

In [12]:
#显示所有列
pd.set_option('display.max_columns', None)
#显示所有行
pd.set_option('display.max_rows', None)
#设置value的显示长度为100，默认为50
pd.set_option('max_colwidth',100)
df.head().append(df.tail())

,SaleID,bodyType,brand,fuelType,gearbox,kilometer,model,notRepairedDamage,power,price,price_boxcox,v_0,v_1,v_10,v_11,v_12,v_13,v_14,v_2,v_3,v_4,v_5,v_6,v_7,v_8,v_9,city,used_time
0,0,1.0,6,0.0,0.0,12.5,30.0,0.0,60.0,1850.0,10.400,43.34,3.967,-2.880,2.8050,-2.420,0.7954,0.9146,0.05026,2.1600,1.1440,0.2357,0.10200,0.12950,0.02281,0.097500,10,4380.1
1,1,2.0,1,0.0,0.0,15.0,40.0,NaN,0.0,3600.0,11.664,45.30,5.234,-4.902,2.0960,-1.030,-1.7230,0.2455,0.13800,1.3810,-1.4220,0.2650,0.12103,0.13570,0.02660,0.020580,43,4745.2
2,2,1.0,15,0.0,0.0,12.5,115.0,0.0,163.0,6222.0,12.760,45.97,4.824,-4.848,1.8040,1.565,-0.8325,-0.2300,1.31900,-0.9985,-0.9970,0.2515,0.11490,0.16520,0.06216,0.027070,28,4319.1
3,3,0.0,10,0.0,1.0,15.0,109.0,0.0,193.0,2400.0,10.880,45.70,4.492,-4.508,1.2860,-0.502,-2.4380,-0.4788,-0.05063,0.8840,-2.2290,0.2744,0.11030,0.12195,0.03340,0.000000,43,7221.0
4,4,1.0,5,0.0,0.0,5.0,110.0,0.0,68.0,5200.0,12.390,44.38,2.031,-1.896,0.9106,0.931,2.8340,1.9240,0.57230,-1.5710,2.2460,0.2280,0.07320,0.09186,0.07880,0.121500,69,1466.4
198985,199995,4.0,4,0.0,0.0,15.0,4.0,0.0,116.0,NaN,NaN,45.62,5.957,-5.977,1.3030,-1.207,-1.9810,-0.3577,-0.91850,0.7750,-2.0210,0.2847,0.13000,0.04984,0.02881,0.004616,32,7233.6
198986,199996,0.0,0,0.0,0.0,15.0,0.0,0.0,75.0,NaN,NaN,43.94,4.477,-3.914,1.7600,-2.076,-1.1550,0.1691,-0.84200,1.3280,-1.2930,0.2680,0.10810,0.06604,0.02547,0.025970,18,6123.4
198987,199997,0.0,1,1.0,1.0,15.0,49.0,0.0,224.0,NaN,NaN,46.53,4.170,-4.640,0.6550,1.138,-1.3910,0.2544,0.38870,-0.7046,-1.4800,0.2695,0.10570,0.11770,0.05746,0.015670,34,4315.9
198988,199998,0.0,1,0.0,1.0,15.0,27.0,0.0,334.0,NaN,NaN,46.78,-3.297,1.834,-2.8280,2.465,-0.9116,-2.0570,0.24350,-1.2770,-0.4048,0.2612,0.00049,0.13730,0.08624,0.051400,19,5122.0
198989,199999,1.0,6,NaN,1.0,9.0,166.0,0.0,68.0,NaN,NaN,43.72,-3.121,2.914,-1.1360,0.548,2.0940,-1.5520,0.02734,-0.8090,2.1170,0.2288,0.00030,0.10350,0.08060,0.124270,32,4295.2


（v1、6、10）（v4、9、13）（v2、5、7）（v3、8）

由于v1与v6相关性0.999，故删去v1；
由于v4与v9相关性0.963，故删去v4；
由于v2与v7先关心0.974，故删去v7；



In [ ]:
df.drop(['v_1','v_4','v_7'], axis=1, inplace=True)

In [14]:
print(df.shape, df.columns)

(198990, 28) Index(['SaleID', 'bodyType', 'brand', 'fuelType', 'gearbox', 'kilometer',
       'model', 'notRepairedDamage', 'power', 'price', 'price_boxcox', 'v_0',
       'v_1', 'v_10', 'v_11', 'v_12', 'v_13', 'v_14', 'v_2', 'v_3', 'v_4',
       'v_5', 'v_6', 'v_7', 'v_8', 'v_9', 'city', 'used_time'],
      dtype='object')


In [18]:
train = df[~df['price'].isnull()].reset_index(drop=True)
test = df[df['price'].isnull()].reset_index(drop=True)

min、max、mean
等距分桶

power:【0, 600, 116】分桶10个：bins = [i*60 for i in range(11)]

kilometer：【0.5, 15, 12.6】   bins = [i*1.5 for i in range(11)]

used_time：【24.5， 9137.3， 4572.3】  bins = [i*920 for i in range(11)]

In [ ]:
# 数据分桶 以 power 为例
# 这时候我们的缺失值也进桶了，
# 为什么要做数据分桶呢，原因有很多，= =
# 1. 离散后稀疏向量内积乘法运算速度更快，计算结果也方便存储，容易扩展；
# 2. 离散后的特征对异常值更具鲁棒性，如 age>30 为 1 否则为 0，对于年龄为 200 的也不会对模型造成很大的干扰；
# 3. LR 属于广义线性模型，表达能力有限，经过离散化后，每个变量有单独的权重，这相当于引入了非线性，能够提升模型的表达能力，加大拟合；
# 4. 离散后特征可以进行特征交叉，提升表达能力，由 M+N 个变量编程 M*N 个变量，进一步引入非线形，提升了表达能力；
# 5. 特征离散后模型更稳定，如用户年龄区间，不会因为用户年龄长了一岁就变化

# 当然还有很多原因，LightGBM 在改进 XGBoost 时就增加了数据分桶，增强了模型的泛化性
bin_col = []
bin_fea = ['power','kilometer','used_time']
sc = [60,1.5,920]
for j in range(3):
    bin = [i*sc[j] for i in range(11)]
    df[bin_fea[j] + '_bin'] = pd.cut(df[bin_fea[j]], bin, labels=False)
    bin_col.append(bin_fea[j] + '_bin')
# 匿名特征也进行分桶，增加特征数量（不确定有没有用，先不做尝试）

In [ ]:
# 查看分桶效果
df[['power_bin','kilometer_bin', 'used_time_bin']].head()

In [ ]:
# 查看特征列，以防上面出错
df.columns

In [ ]:
cate_fea = ['bodyType','brand','fuelType','gearbox','model','notRepairedDamage','power_bin','kilometer_bin', 'used_time_bin']

## 类别特征的目标编码

In [ ]:
from sklearn.model_selection import KFold
from tqdm import tqdm

train_df = df[~df['price'].isnull()].reset_index(drop=True)
test_df = df[df['price'].isnull()].reset_index(drop=True)


### target encoding目标编码，回归场景相对来说做目标编码的选择更多，不仅可以做均值编码，还可以做标准差编码、中位数编码等
enc_cols = []
stats_default_dict = {
    'max': train_df['price'].max(),
    'min': train_df['price'].min(),
    'median': train_df['price'].median(),
    'mean': train_df['price'].mean(),
    'sum': train_df['price'].sum(),
    'std': train_df['price'].std(),
    'skew': train_df['price'].skew(),
    'kurt': train_df['price'].kurt(),
    'mad': train_df['price'].mad()
}
### 暂且选择这几种编码（可以选择完）
enc_stats = ['max','min','mean','median','sum','std','mad','skew','kurt']
skf = KFold(n_splits=5, shuffle=True, random_state=2020)
# 遍历类别特征（加上分桶的）
for f in tqdm(cate_fea):
    enc_dict = {}
    for stat in enc_stats:
        enc_dict['{}_target_{}'.format(f, stat)] = stat
        train_df['{}_target_{}'.format(f, stat)] = 0
        test_df['{}_target_{}'.format(f, stat)] = 0
        enc_cols.append('{}_target_{}'.format(f, stat))
    # skf.split()返回索引的迭代器        
    for i, (trn_idx, val_idx) in enumerate(skf.split(train_df, train_df['price'])):
        trn_x, val_x = train_df.iloc[trn_idx].reset_index(drop=True), train_df.iloc[val_idx].reset_index(drop=True)
        enc_df = trn_x.groupby(f, as_index=False)['price'].agg(enc_dict)
        val_x = val_x[[f]].merge(enc_df, on=f, how='left')
        test_x = test_df[[f]].merge(enc_df, on=f, how='left')
        for stat in enc_stats:
            val_x['{}_target_{}'.format(f, stat)] = val_x['{}_target_{}'.format(f, stat)].fillna(stats_default_dict[stat])
            test_x['{}_target_{}'.format(f, stat)] = test_x['{}_target_{}'.format(f, stat)].fillna(stats_default_dict[stat])
            train_df.loc[val_idx, '{}_target_{}'.format(f, stat)] = val_x['{}_target_{}'.format(f, stat)].values
            test_df['{}_target_{}'.format(f, stat)] += test_x['{}_target_{}'.format(f, stat)].values / skf.n_splits


cols = cate_fea + num_fea + enc_cols
sub = test_df[['SaleID']].copy()
test_df = test_df[cols]
labels = train_df['price'].values
train_df = train_df[cols]
print(train_df.shape)
train_df.head()

In [ ]:
# 查看缺失值信息
train_df.info()
# test_df.info()
# 显碍眼不看那么多

In [ ]:
# 由于特征构造不允许空值，故这里对其进行填充
# 方法这里采用中位数、也可以用模型、插值法、KNN等，有时间再慢慢试
train_df.fillna(train.median(),inplace=True)
test_df.fillna(test.median(), inplace=True)

## 数值特征处理

In [ ]:
# 把数值特征取出来
num_fea = ['kilometer','power','used_time'] + ['v_{}'.format(i+8) for i in range(7)]+ ['v_0','v_2','v_3','v_5','v_6']

In [ ]:
# 导入构造多项式的包，这里构造的是二阶
from sklearn.preprocessing import PolynomialFeatures
# 这里没有选择全部数字特征，根据相关性选择的，有时间可以选择完试试
rest_df = train_df[['kilometer','power','used_time','v_0','v_2','v_5','v_6','v_9','v_10','v_11','v_12','v_14']]
rest_df1 = test_df[['kilometer','power','used_time','v_0','v_2','v_5','v_6','v_9','v_10','v_11','v_12','v_14']]
# 打错了，不想删了
col1 = ['kilometer','power','used_time','v_0','v_2','v_5','v_6','v_9','v_10','v_11','v_12','v_14']

In [ ]:
# 训练集的
poly_transform = PolynomialFeatures(degree=2,interaction_only=False,include_bias=False)
poly_features = pd.DataFrame(poly_transform.fit_transform(rest_df), columns=poly_transform.get_feature_names(col1))
# for col in poly_features.columns:
for col in poly_features.columns:
    if col not in col1:
        train_df.insert(1,col,poly_features[col])

In [ ]:
# 测试集的，也可以合并处理
poly_transform = PolynomialFeatures(degree=2,interaction_only=False,include_bias=False)
poly_features = pd.DataFrame(poly_transform.fit_transform(rest_df1), columns=poly_transform.get_feature_names(col1))
# for col in poly_features.columns:
for col in poly_features.columns:
    if col not in col1:
        test_df.insert(1,col,poly_features[col])

In [ ]:
# 查看构造完特征的纬度
train_df.columns

### 删除类别特征，在进行筛选特征

In [ ]:
from sklearn.model_selection import KFold
from lightgbm.sklearn import LGBMRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
import time


oof = np.zeros(train_df.shape[0])
sub['price'] = 0
feat_imp_df = pd.DataFrame({'feat': train_df.columns, 'imp': 0})
skf = KFold(n_splits=5, shuffle=True, random_state=2020)
### 参数可以重新设置一下，天池这里面太慢了所以随便跑跑
clf = LGBMRegressor(
    learning_rate=0.1,
    n_estimators=1000,
    num_leaves=31,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=2020,
    metric=None
)
for i, (trn_idx, val_idx) in enumerate(skf.split(train_df, labels)):
    print('--------------------- {} fold ---------------------'.format(i))
    t = time.time()
    trn_x, trn_y = train_df.iloc[trn_idx].reset_index(drop=True), labels[trn_idx]
    val_x, val_y = train_df.iloc[val_idx].reset_index(drop=True), labels[val_idx]
    clf.fit(
        trn_x, trn_y,
        eval_set=[(val_x, val_y)],
        categorical_feature=cate_fea,
        eval_metric='mae',
        early_stopping_rounds=100,
        verbose=200
    )
    feat_imp_df['imp'] += clf.feature_importances_ / skf.n_splits
#     feat_imp_df['imp'] += clf.feature_importances_ 
    oof[val_idx] = clf.predict(val_x)
    sub['price'] += clf.predict(test_df) / skf.n_splits
    print('val mse:', mean_squared_error(val_y, oof[val_idx]))
    print('runtime: {}\n'.format(time.time() - t))

mae = mean_absolute_error(labels, oof)
mse = mean_squared_error(labels, oof)
print('cv mae:', mae)
print('cv mse:', mse)
print('sub mean:', sub['price'].mean())
# sub.to_csv('sub_{}_{}_{}.csv'.format(mae, mse, sub['price'].mean()), index=False)

In [ ]:
sns.distplot(labels, label='train', color='y', hist=False)
sns.distplot(oof, label='oof', color='g', hist=False)
sns.distplot(sub['price'], label='test', color='r', hist=False)

In [ ]:
plt.figure(figsize=(15, 30))
feat_imp_df = feat_imp_df.sort_values('imp').reset_index(drop=True)
f = sns.barplot(x='imp', y='feat', data=feat_imp_df)

In [ ]:
a = feat_imp_df.sort_values('imp',ascending=False)

In [ ]:
a['imp'] = a['imp']/(a['imp'].sum())

In [ ]:
# 为了画概率图，不知道用那个函数？
b = []
c = 0
for i in range(132):
    c = (a['imp'][131 - i] + c)
    b.append(c)

In [ ]:
plt.figure(figsize=(10,8))
plt.plot(range(1,133,1),b,'ro')

## 归一化处理，查看是否删除类别特征

## 模型训练与测试

In [ ]:
def Sta_inf(data):
    print('_min',np.min(data))
    print('_max:',np.max(data))
    print('_mean',np.mean(data))
    print('_ptp',np.ptp(data))
    print('_std',np.std(data))
    print('_var',np.var(data))

In [ ]:
X_data = train_df
Y_data = train['price']
X_test = test_df

调参，五折交叉验证

In [ ]:
import lightgbm as lgb
import xgboost as xgb

## 参数搜索和评价的
from sklearn.model_selection import GridSearchCV,cross_val_score,StratifiedKFold,train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
## xgb-Model
xgr = xgb.XGBRegressor(n_estimators=120, learning_rate=0.1, gamma=0, subsample=0.8,\
        colsample_bytree=0.9, max_depth=7) #,objective ='reg:squarederror'

scores_train = []
scores = []

## 5折交叉验证方式
sk=StratifiedKFold(n_splits=5,shuffle=True,random_state=0)
for train_ind,val_ind in sk.split(X_data,Y_data):
    
    train_x=X_data.iloc[train_ind].values
    train_y=Y_data.iloc[train_ind]
    val_x=X_data.iloc[val_ind].values
    val_y=Y_data.iloc[val_ind]
    
    xgr.fit(train_x,train_y)
    pred_train_xgb=xgr.predict(train_x)
    pred_xgb=xgr.predict(val_x)
    
    score_train = mean_absolute_error(train_y,pred_train_xgb)
    scores_train.append(score_train)
    score = mean_absolute_error(val_y,pred_xgb)
    scores.append(score)

print('Train mae:',np.mean(score_train))
print('Val mae',np.mean(scores))

In [ ]:
def build_model_xgb(x_train,y_train):
    model = xgb.XGBRegressor(n_estimators=150, learning_rate=0.1, gamma=0, subsample=0.8,\
        colsample_bytree=0.9, max_depth=7) #, objective ='reg:squarederror'
    model.fit(x_train, y_train)
    return model

def build_model_lgb(x_train,y_train):
    estimator = lgb.LGBMRegressor(num_leaves=127,n_estimators = 150)
    param_grid = {
        'learning_rate': [0.01, 0.05, 0.1, 0.2],
    }
    gbm = GridSearchCV(estimator, param_grid)
    gbm.fit(x_train, y_train)
    return gbm

In [ ]:
## Split data with val
x_train,x_val,y_train,y_val = train_test_split(X_data,Y_data,test_size=0.3)

In [ ]:
print('Train lgb...')
model_lgb = build_model_lgb(x_train,y_train)
val_lgb = model_lgb.predict(x_val)
MAE_lgb = mean_absolute_error(y_val,val_lgb)
print('MAE of val with lgb:',MAE_lgb)

print('Predict lgb...')
model_lgb_pre = build_model_lgb(X_data,Y_data)
subA_lgb = model_lgb_pre.predict(X_test)
print('Sta of Predict lgb:')
Sta_inf(subA_lgb)

In [ ]:
print('Train xgb...')
model_xgb = build_model_xgb(x_train,y_train)
val_xgb = model_xgb.predict(x_val)
MAE_xgb = mean_absolute_error(y_val,val_xgb)
print('MAE of val with xgb:',MAE_xgb)

print('Predict xgb...')
model_xgb_pre = build_model_xgb(X_data,Y_data)
subA_xgb = model_xgb_pre.predict(X_test)
print('Sta of Predict xgb:')
Sta_inf(subA_xgb)

In [ ]:
## 这里我们采取了简单的加权融合的方式
val_Weighted = (1-MAE_lgb/(MAE_xgb+MAE_lgb))*val_lgb+(1-MAE_xgb/(MAE_xgb+MAE_lgb))*val_xgb
val_Weighted[val_Weighted<0]=10 # 由于我们发现预测的最小值有负数，而真实情况下，price为负是不存在的，由此我们进行对应的后修正
print('MAE of val with Weighted ensemble:',mean_absolute_error(y_val,val_Weighted))

In [ ]:
sub_Weighted = (1-MAE_lgb/(MAE_xgb+MAE_lgb))*subA_lgb+(1-MAE_xgb/(MAE_xgb+MAE_lgb))*subA_xgb

## 查看预测值的统计进行
plt.hist(Y_data)
plt.show()
plt.close()

In [ ]:
# 查看数据预测结果
sub_Weighted

In [ ]:
sub = pd.DataFrame()
sub['SaleID'] = test.SaleID
sub['price'] = sub_Weighted
sub.to_csv('sub1.csv',index=False)

# 时间关系代码运行结果不展示了